In [2]:
# TODO: Dodać komentarze do kodu i zrobić jego refactor
# W przód
struct Dual{T <:Number} <:Number
    v::T
    dv::T
end

In [3]:
import Base: +, -, *, /
     -(x::Dual) = Dual(-x.v, -x.dv)
     +(x::Dual, y::Dual) = Dual( x.v + y.v, x.dv + y.dv)
     -(x::Dual, y::Dual) = Dual( x.v - y.v, x.dv - y.dv)
     *(x::Dual, y::Dual) = Dual( x.v * y.v, x.dv * y.v + x.v * y.dv)
     /(x::Dual, y::Dual) = Dual( x.v / y.v, (x.dv * y.v - x.v * y.dv)/y.v^2)
import Base: abs, sin, cos, tan, exp, sqrt, isless
    abs(x::Dual) = Dual(abs(x.v),sign(x.v)*x.dv)
    sin(x::Dual) = Dual(sin(x.v), cos(x.v)*x.dv)
    cos(x::Dual) = Dual(cos(x.v),-sin(x.v)*x.dv)
    tan(x::Dual) = Dual(tan(x.v), one(x.v)*x.dv + tan(x.v)^2*x.dv)
    exp(x::Dual) = Dual(exp(x.v), exp(x.v)*x.dv)
    sqrt(x::Dual) = Dual(sqrt(x.v),.5/sqrt(x.v) * x.dv)
    isless(x::Dual, y::Dual) = x.v < y.v;

In [4]:
import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");
value(x::Dual) = x.v;
partials(x::Dual) = x.dv;

In [5]:
import Base: convert, promote_rule
convert(::Type{Dual{T}}, x::Dual) where T =
 Dual(convert(T, x.v), convert(T, x.dv))
convert(::Type{Dual{T}}, x::Number) where T =
 Dual(convert(T, x), zero(T))
promote_rule(::Type{Dual{T}}, ::Type{R}) where {T,R} =
 Dual{promote_type(T,R)}

promote_rule (generic function with 125 methods)

In [8]:
# TODO: Przetestować dla funkcji ReLU, soft-max, podstawowych funkcji matematycznych
f(x) = sin(x*x)
ϵ = Dual(0., 1.)
x = 5.0 + ϵ
y = f(x)

(-0.13235175009777303) + [9.912028118634735ϵ]

In [20]:
# TODO: Ogarnąć wyznaczenie macierzy jakobiego
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
            Dual(args[j], one(args[j])) :
            Dual(args[j],zero(args[j])) )
        end
        column = partials.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

In [26]:
#przykładowe funkcje ReLu i soft max i sigma
ReLu(x) = max(zero(x),x)
σ(x) =  one(x)/(one(x)+exp(-x))
softmaxx(x) = exp.(x) ./ sum(exp.(x));
g(x) = x[1]*x[2]
wynik = J(softmaxx, [1.0,2.0])
print(wynik)


[0.19661193324148188 -0.19661193324148188; -0.19661193324148188 0.19661193324148188]

In [8]:
# TODO: potestować metodę (w tym dla funkcji Rosenbrocka), zapisać wyniki

In [2]:
# w tył
struct Operator{T}
    f::T
end

abstract type Node end
abstract type LeafNode <: Node end

mutable struct Variable{T} <: LeafNode
    value::T
    grad::T
    
    Variable(val::T) where T = new{T}(val, zero(val))
    Variable(val::T, grad::T) where T = new{T}(val, grad)
end

struct ComputableNode{OT, AT} <: Node
    op::OT # operacja jaka będzie wykonywana na node
    args::AT # argumenty operacji
end

mutable struct CachedNode{NT, OUT} <: Node
    node::NT # ComputableNode
    out::OUT # wynik operacji na zadanych argumentach w computable Node
end

In [3]:
# rejestracja operacji z wyrażenia wejściowego
function register(op, args...)
    concreteOp = Operator(op)
    node = ComputableNode(concreteOp, args)
    out = forward(node)
    CachedNode(node, out)
end

import Base: +, -, *, /
    -(x::Node) = register(-, x)
    +(x::Node, y::Node) = register(+, x, y)
    -(x::Node, y::Node) = register(-, x, y)
    *(x::Node, y::Node) = register(*, x, y)
    /(x::Node, y::Node) = register(/, x, y)
import Base: abs, sin, cos, tan, exp, sqrt, isless, zero, one
    abs(x::Node) = register(abs, x)
    sin(x::Node) = register(sin, x)
    cos(x::Node) = register(cos, x)
    tan(x::Node) = register(tan, x)
    exp(x::Node) = register(exp, x)
    sqrt(x::Node) = register(sqrt, x)
    isless(x::Node, y::Node) = register(isless, x, y)
    zero(x::Node) = register(zero, x)
    one(x::Node) = register(one, x)

one (generic function with 20 methods)

In [4]:
forward(cached::CachedNode) = cached.out = forward(cached.node)
forward(node::ComputableNode) = forward(node.op, map(forward, node.args)...) # dokonujemy mapowania funkcją forward aby dobrac się do value zmiennych variable
forward(op::Operator, args...) = op.f(args...)  # finalne wykonanie operacji na zadanych argumentach i obliczenie wyniku
forward(var::Variable) = var.value # wykorzystywane do mapowania

forward (generic function with 4 methods)

In [5]:
function backward(cached::CachedNode, grad)
    grad_inputs = gradient(cached, grad)
    for (each, each_grad) in zip(cached.node.args, grad_inputs)
        backward(each, each_grad)
    end
end

gradient(cached::CachedNode, grad) =
    gradient(cached.node.op, grad, map(value, cached.node.args)...)

gradient(op::Operator, grad, args...) =
    gradient(op.f, grad, args...)

value(cached::CachedNode) = cached.out
value(var::Variable) = var.value

gradient(::typeof(-), grad, x) = (-grad, )
gradient(::typeof(+), grad, x, y) = (grad, grad)
gradient(::typeof(-), grad, x, y) = (grad, -grad)
gradient(::typeof(*), grad, x, y) = (grad * y, grad * x) # (pochodna po x, pochodna po y)
gradient(::typeof(/), grad, x, y) = (grad * 1/y, grad * -(x / (y * y)))
gradient(::typeof(abs), grad, x) = (grad * x/abs(x), ) 
gradient(::typeof(sin), grad, x) = (grad * cos(x), ) # dodajemy ',' aby cały czas to był tuple
gradient(::typeof(cos), grad, x) = (grad * -sin(x), )
gradient(::typeof(tan), grad, x) = (grad * (tan(x)*tan(x)+1), )
gradient(::typeof(exp), grad, x) = (grad * exp(x), )
gradient(::typeof(sqrt), grad, x) = (grad * 1/(2*sqrt(x)), )
gradient(::typeof(isless), grad, x, y) = (isless(x, y) ? grad * one(x) : grad * zero(x), 
                                          isless(x, y) ? grad * zero(y) : grad * one(y))
gradient(::typeof(zero), grad, x) = (grad * zero(x), )
gradient(::typeof(one), grad, x) = (grad * one(x), )

function backward(var::Variable, grad)
    var.grad += grad
    #println(var)    
end

backward (generic function with 2 methods)

In [6]:
# TODO: Przetestować dla funkcji ReLU, soft-max, podstawowych funkcji matematycznych
f(x) = zero(x)
ReLu(x) = max(zero(x),x)
x = [1]
println(ReLu(x))
σ(x) =  one(x)/(one(x)+exp(-x))
softmaxx(x) = exp.(x) ./ sum(exp.(x));
x = [Variable(5.0, 0.0), Variable(6.0, 0.0)]
y = f(x)
backward(y, 1.0)
println(y) #aby dobrać się do gradientów trzeba przejść po grafie i wyszukać gradienty

[1]


LoadError: MethodError: no method matching zero(::Type{Variable{Float64}})
[0mClosest candidates are:
[0m  zero([91m::Union{Type{P}, P}[39m) where P<:Dates.Period at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\Dates\src\periods.jl:53
[0m  zero([91m::AbstractIrrational[39m) at irrationals.jl:148
[0m  zero([91m::Node[39m) at In[3]:23
[0m  ...

In [7]:
# checking results
using ForwardDiff
h(x) = sin(x[1]*x[2])
ReLu(x) = max(zero(x),x)
sigma(x) =  one(x)/(one(x)+exp(-x))
softmaxx(x) = exp.(x) ./ sum(exp.(x));
x = [5.0, 6.0]
@show ForwardDiff.jacobian(sigma,x)

LoadError: MethodError: no method matching one(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{typeof(sigma), Float64}, Float64, 2}})
[0mClosest candidates are:
[0m  one([91m::Union{Type{T}, T}[39m) where T<:AbstractString at strings/basic.jl:262
[0m  one([91m::Union{Type{P}, P}[39m) where P<:Dates.Period at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\Dates\src\periods.jl:54
[0m  one([91m::Node[39m) at In[3]:24
[0m  ...

In [ ]:
# EXAMPLE OF USING JULIA DIFF
using ForwardDiff
h(x) = sin(x[1]) + x[1] * x[2] + sinh(x[1] * x[2]) # multivariate.
x = [1.4 2.2]
@show ForwardDiff.gradient(h,x) # use AD, seeds from x

#Or, can use complicated functions of many variables
f(x) = sum(sin, x) + prod(tan, x) * sum(sqrt, x)
g = (x) -> ForwardDiff.gradient(f, x); # g() is now the gradient
g(rand(5)) # gradient at a random point
# ForwardDiff.hessian(f,x') # or the hessian

In [ ]:
#TODO: macierze Jakobiego dla różniczkowania w tył

In [ ]:
# TODO: potestować metodę (w tym dla funkcji Rosenbrocka), zapisać wyniki

In [ ]:
# TODO: Zrobić krótkie zestawienie metod